In [1]:
# Library Imports.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Allows plots to appear directly in the notebook.
%matplotlib inline

from patsy import dmatrices
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import export_graphviz
import pydot

import pickle
import math

In [2]:
df = pd.read_csv('full_dataframe.csv', index_col=0)
df.dtypes

/Users/Stephen/anaconda3/envs/dublinbikes/lib/python3.7/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


number                int64
name                 object
status               object
bike_stands           int64
available_bikes       int64
available_stands      int64
last_update          object
banking               int64
weather_id            int64
main                 object
description          object
temperature         float64
humidity            float64
wind_speed          float64
dtype: object

In [3]:
df.dtypes

number                int64
name                 object
status               object
bike_stands           int64
available_bikes       int64
available_stands      int64
last_update          object
banking               int64
weather_id            int64
main                 object
description          object
temperature         float64
humidity            float64
wind_speed          float64
dtype: object

In [4]:
df['description'].value_counts()

broken clouds                   332210
light rain                      195687
scattered clouds                141655
few clouds                      120134
moderate rain                    79911
clear sky                        73475
shower rain                      63527
light intensity shower rain      52224
mist                             32949
heavy intensity rain             19857
light intensity drizzle          16280
fog                              14763
light intensity drizzle rain      4232
very heavy rain                   3052
snow                              1458
shower sleet                      1140
overcast clouds                    240
Name: description, dtype: int64

In [5]:
df['main'].value_counts()

Clouds     594239
Rain       414258
Clear       73475
Mist        32949
Drizzle     20512
Fog         14763
Snow         2598
Name: main, dtype: int64

In [6]:
for col in df[['description']].columns:
    if [df['description']=='clear sky']:
        df['description'].replace('clear sky',1, inplace=True)
    if[df['description']=='few clouds']:
        df['description'].replace('few clouds', 2, inplace=True)
    if[df['description']=='scattered clouds']:
        df['description'].replace('scattered clouds',3, inplace=True)
    if [df['description']=='broken clouds']:
        df['description'].replace('broken clouds',4, inplace=True)
    if [df['description']=='fog']:
        df['description'].replace('fog',5, inplace=True)
    if [df['description']=='mist']:
        df['description'].replace('mist',6, inplace=True)
    if [df['description']=='light intensity drizzle']:
        df['description'].replace('light intensity drizzle',7, inplace=True)
    if [df['description']=='light intensity drizzle rain']:
        df['description'].replace('light intensity drizzle rain',8, inplace=True)
    if [df['description']=='light intensity shower rain']:
        df['description'].replace('light intensity shower rain',9, inplace=True)
    if [df['description']=='shower rain']:
        df['description'].replace('shower rain',10, inplace=True)
    if [df['description']=='heavy intensity rain']:
        df['description'].replace('heavy intensity rain',11, inplace=True)
    if [df['description']=='light rain']:
        df['description'].replace('light rain',12, inplace=True)
    if [df['description']=='moderate rain']:
        df['description'].replace('moderate rain',13, inplace=True)
    if [df['description']=='very heavy rain']:
        df['description'].replace('very heavy rain',14, inplace=True)
    if [df['description']=='shower sleet']:
        df['description'].replace('shower sleet',15, inplace=True)
    if [df['description']=='snow']:
        df['description'].replace('snow',16, inplace=True)
    if [df['description']=='overcast clouds']:
        df['description'].replace('overcast clouds',17, inplace=True)

In [7]:
df['description'].value_counts()

4     332210
12    195687
3     141655
2     120134
13     79911
1      73475
10     63527
9      52224
6      32949
11     19857
7      16280
5      14763
8       4232
14      3052
16      1458
15      1140
17       240
Name: description, dtype: int64

In [8]:
df['last_update'] = df['last_update'].astype('datetime64[ns]')

In [9]:
df['week'] = df['last_update'].dt.week

In [10]:
df['day'] = df['last_update'].dt.weekday

In [11]:
df.dtypes

number                       int64
name                        object
status                      object
bike_stands                  int64
available_bikes              int64
available_stands             int64
last_update         datetime64[ns]
banking                      int64
weather_id                   int64
main                        object
description                  int64
temperature                float64
humidity                   float64
wind_speed                 float64
week                         int64
day                          int64
dtype: object

In [12]:
df['last_update'] = df['last_update'].dt.round('30min') 

In [13]:
df['time'] =df.last_update.map(lambda t: t.strftime('%H%M'))

In [14]:
df = df.drop_duplicates()

In [15]:
df.dtypes

number                       int64
name                        object
status                      object
bike_stands                  int64
available_bikes              int64
available_stands             int64
last_update         datetime64[ns]
banking                      int64
weather_id                   int64
main                        object
description                  int64
temperature                float64
humidity                   float64
wind_speed                 float64
week                         int64
day                          int64
time                        object
dtype: object

In [16]:
df = df.drop(['last_update', 'number', 'banking', 'bike_stands', 'weather_id', 'main', 'temperature', 'humidity', 'wind_speed'] , axis=1)

In [17]:
df = df.loc[(df['name'] == 'GRAND CANAL DOCK')]

In [18]:
df.reset_index(drop=True)

,name,status,available_bikes,available_stands,description,week,day,time
0,GRAND CANAL DOCK,OPEN,28,12,4,8,3,1700
1,GRAND CANAL DOCK,OPEN,27,13,4,8,3,1700
2,GRAND CANAL DOCK,OPEN,26,14,4,8,3,1700
3,GRAND CANAL DOCK,OPEN,24,16,4,8,3,1700
4,GRAND CANAL DOCK,OPEN,22,18,4,8,3,1700
5,GRAND CANAL DOCK,OPEN,23,17,4,8,3,1700
6,GRAND CANAL DOCK,OPEN,25,15,4,8,3,1700
7,GRAND CANAL DOCK,OPEN,26,14,4,8,3,1730
8,GRAND CANAL DOCK,OPEN,27,13,4,8,3,1730
9,GRAND CANAL DOCK,OPEN,29,11,4,8,3,1730


In [19]:
df.dtypes

name                object
status              object
available_bikes      int64
available_stands     int64
description          int64
week                 int64
day                  int64
time                object
dtype: object

In [20]:
# df.groupby('time')['available_bikes'].mean()

In [21]:
# df['mean_available_bikes'] = df.groupby('time')['available_bikes'].transform('mean').astype(int)

In [22]:
df.dtypes

name                object
status              object
available_bikes      int64
available_stands     int64
description          int64
week                 int64
day                  int64
time                object
dtype: object

In [23]:
df

,name,status,available_bikes,available_stands,description,week,day,time
85,GRAND CANAL DOCK,OPEN,28,12,4,8,3,1700
132,GRAND CANAL DOCK,OPEN,27,13,4,8,3,1700
448,GRAND CANAL DOCK,OPEN,26,14,4,8,3,1700
500,GRAND CANAL DOCK,OPEN,24,16,4,8,3,1700
530,GRAND CANAL DOCK,OPEN,22,18,4,8,3,1700
606,GRAND CANAL DOCK,OPEN,23,17,4,8,3,1700
772,GRAND CANAL DOCK,OPEN,25,15,4,8,3,1700
1046,GRAND CANAL DOCK,OPEN,26,14,4,8,3,1730
1060,GRAND CANAL DOCK,OPEN,27,13,4,8,3,1730
1119,GRAND CANAL DOCK,OPEN,29,11,4,8,3,1730


In [24]:
#new_df = new_df.drop_duplicates(subset=['time'], keep='first')

In [25]:
# new_df = new_df.sort_values(by=['mean_available_bikes'])

In [26]:
df

,name,status,available_bikes,available_stands,description,week,day,time
85,GRAND CANAL DOCK,OPEN,28,12,4,8,3,1700
132,GRAND CANAL DOCK,OPEN,27,13,4,8,3,1700
448,GRAND CANAL DOCK,OPEN,26,14,4,8,3,1700
500,GRAND CANAL DOCK,OPEN,24,16,4,8,3,1700
530,GRAND CANAL DOCK,OPEN,22,18,4,8,3,1700
606,GRAND CANAL DOCK,OPEN,23,17,4,8,3,1700
772,GRAND CANAL DOCK,OPEN,25,15,4,8,3,1700
1046,GRAND CANAL DOCK,OPEN,26,14,4,8,3,1730
1060,GRAND CANAL DOCK,OPEN,27,13,4,8,3,1730
1119,GRAND CANAL DOCK,OPEN,29,11,4,8,3,1730


## Preparing the columns

In [27]:
item_list = list(range(df.shape[1]))
list_to_remove = [0,1,2,3,5,6]
final_list= list(set(item_list).difference(set(list_to_remove)))
print(final_list)

[4, 7]


In [28]:
X = df.iloc[:, final_list].values
y = df.iloc[:, 2].values

## Splitting to train and test set & Creating Model

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=False) 

In [30]:
sc = StandardScaler()  
X_train = sc.fit_transform(X_train)  
X_test = sc.transform(X_test)

/Users/Stephen/anaconda3/envs/dublinbikes/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/Stephen/anaconda3/envs/dublinbikes/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/Stephen/anaconda3/envs/dublinbikes/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [31]:
# Running the X_train and y_train on the RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=0)  
model.fit(X_train, y_train)  

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

## Model Parameters

In [32]:
model_score = model.score(X_train, y_train)

In [33]:
print("Co-efficients of determination R^2 of prediction: ", model_score)

Co-efficients of determination R^2 of prediction:  0.5590013098425772


## Testing the model on X_train

In [34]:
y_pred = model.predict(X_train)  

In [35]:
print('MAE:', metrics.mean_absolute_error(y_train, y_pred))  
print('MSE:', metrics.mean_squared_error(y_train, y_pred))  
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_train, y_pred)))
print('R2: ', metrics.r2_score(y_train,y_pred))

MAE: 6.929447131388891
MSE: 85.11491010557825
RMSE: 9.22577422797557
R2:  0.5590013098425772


## Testing the model on X_test

In [36]:
y_pred = model.predict(X_test)  

In [37]:
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))  
print('MSE:', metrics.mean_squared_error(y_test, y_pred))  
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R2: ', metrics.r2_score(y_test,y_pred))

MAE: 7.420413188634935
MSE: 95.45199442140304
RMSE: 9.769953655028413
R2:  0.496134084915139


In [38]:
print('Test Variance score: %.2f' % r2_score(y_test, y_pred))

Test Variance score: 0.50


## Visualising

In [40]:
labels = np.array(df['available_bikes'])

In [41]:
df.columns

Index(['name', 'status', 'available_bikes', 'available_stands', 'description',
       'week', 'day', 'time'],
      dtype='object')

In [42]:
df = df.drop(['name', 'status', 'available_bikes', 'available_stands', 'week', 'day'], axis=1)

In [43]:
feature_list = list(df)

In [44]:
tree = model.estimators_[5]

In [45]:
export_graphviz(tree, out_file = 'tree.dot', feature_names = feature_list, rounded = True, precision = 1)

In [46]:
(graph, ) = pydot.graph_from_dot_file('tree.dot')

In [47]:
graph.write_png('tree.png')

## Cross-validation testing

In [48]:
scoring = {'abs_error': 'neg_mean_absolute_error', 'squared_error': 'neg_mean_squared_error', 'r2':'r2'}

scores = cross_validate(model,X_test, y_test, cv=10, scoring=scoring, return_train_score=True)
print("MAE :" , abs(scores['test_abs_error'].mean()), "RMSE :", math.sqrt(abs(scores['test_squared_error'].mean())), "| r2 :", scores['test_r2'].mean())

MAE : 7.68488786221277 RMSE : 10.27861294539218 | r2 : 0.43918575992987857


In [49]:
# Looking at the co-efficient importance for each feature
print("Features and co-efficients: " , list(zip(['weather','time'], model.feature_importances_)))

Features and co-efficients:  [('weather', 0.15327621379610945), ('time', 0.8467237862038905)]


## Pickle the model

In [50]:
pickle_model = 'GRAND_CANAL_STATION_model.sav'

In [51]:
pickle.dump(model, open(pickle_model, 'wb'))

In [52]:
# loaded_model = pickle.load(open(pickle_model, 'rb'))
# result = loaded_model.score(pass_test_here)
# print(result)

## Testing Model

In [53]:
model.predict([[8,'1400']])

array([8.08462374])